## WorldClim Attribution

In [2]:
import os
import arcpy
import exactextract
from tqdm import tqdm
import gc

In [3]:
arcpy.env.overwriteOutput = True

In [4]:
# Path to WorldClim's bioclimatic rasters. Downloaded from: https://www.worldclim.org/data/worldclim21.html
wc_folder = r"R:\FWL\Arismendi-Lab\Andres\Gilbert_Freshwater_Fish_Analysis\Environmental_Data\WorldClim"
# Path to feature class with species ranges
input_ranges = "Global_Grid"
# Directory for intermediate processing file
temp_folder = r"D:\Andres\Dam_Project_D\scratch"

In [5]:
# create list with land cover rasters 
with arcpy.EnvManager(workspace=wc_folder):
    wc_rasters = arcpy.ListRasters()

wc_rasters # check list

['wc2.1_30s_bio_1.tif',
 'wc2.1_30s_bio_10.tif',
 'wc2.1_30s_bio_11.tif',
 'wc2.1_30s_bio_12.tif',
 'wc2.1_30s_bio_13.tif',
 'wc2.1_30s_bio_14.tif',
 'wc2.1_30s_bio_15.tif',
 'wc2.1_30s_bio_16.tif',
 'wc2.1_30s_bio_17.tif',
 'wc2.1_30s_bio_18.tif',
 'wc2.1_30s_bio_19.tif',
 'wc2.1_30s_bio_2.tif',
 'wc2.1_30s_bio_3.tif',
 'wc2.1_30s_bio_4.tif',
 'wc2.1_30s_bio_5.tif',
 'wc2.1_30s_bio_6.tif',
 'wc2.1_30s_bio_7.tif',
 'wc2.1_30s_bio_8.tif',
 'wc2.1_30s_bio_9.tif']

In [6]:
# isolate names of variables of interest based on source files convention
wc_fields = ["bio_" + wc_ras.split("_")[3].replace(".tif","") for wc_ras in wc_rasters]
wc_fields

['bio_1',
 'bio_10',
 'bio_11',
 'bio_12',
 'bio_13',
 'bio_14',
 'bio_15',
 'bio_16',
 'bio_17',
 'bio_18',
 'bio_19',
 'bio_2',
 'bio_3',
 'bio_4',
 'bio_5',
 'bio_6',
 'bio_7',
 'bio_8',
 'bio_9']

In [6]:
add_fields_str = " FLOAT; ".join(wc_fields) + " FLOAT"
add_fields_str

'bio_1 FLOAT; bio_10 FLOAT; bio_11 FLOAT; bio_12 FLOAT; bio_13 FLOAT; bio_14 FLOAT; bio_15 FLOAT; bio_16 FLOAT; bio_17 FLOAT; bio_18 FLOAT; bio_19 FLOAT; bio_2 FLOAT; bio_3 FLOAT; bio_4 FLOAT; bio_5 FLOAT; bio_6 FLOAT; bio_7 FLOAT; bio_8 FLOAT; bio_9 FLOAT'

In [7]:
# add fields to feature class to collect summary stats
arcpy.management.AddFields(input_ranges, add_fields_str)

<Result 'Global_Grid'>

In [7]:
# create temporary SQLite geopackage for better compatibility with exactextract library
arcpy.management.CreateSQLiteDatabase(
    out_database_name=fr"{temp_folder}\sqlite.gpkg",
    spatial_type="GEOPACKAGE"
)

<Result 'D:\\Andres\\Dam_Project_D\\scratch\\sqlite.gpkg'>

In [8]:
# not worried about projecting the layers, as exactextract's area-weighting is based on pixel counts

fields = ['OBJECTID'] + wc_fields

with arcpy.da.UpdateCursor(input_ranges, fields) as cursor:
    
    for row in tqdm(cursor,total=int(arcpy.management.GetCount(input_ranges)[0])):
        
        if None not in row[1:]:
            continue
            
        currRange=fr"{temp_folder}\sqlite.gpkg\currRange"

        with arcpy.EnvManager(outputCoordinateSystem=fr"{wc_folder}\{wc_rasters[0]}"):
            arcpy.conversion.ExportFeatures(
                in_features=input_ranges,
                out_features=currRange,
                where_clause=f"OBJECTID = {row[0]}",
                field_mapping=fr'sci_name "sci_name" true true false 100 Text 0 0,First,#,{input_ranges},sci_name,0,99',
    
            )
        
        for wc_ras in wc_rasters:
            
            currStats = exactextract.exact_extract(rast=fr"{wc_folder}\{wc_ras}",
                                                   vec=currRange,
                                                   ops="mean",
                                                   include_cols="sci_name",
                                                   include_geom=False,
                                                   output='pandas'
                                                  )
            
            currField = "bio_" + wc_ras.split("_")[3].replace(".tif","")
            
            row[fields.index(currField)] = currStats['mean'][0]

            cursor.updateRow(row)

            # an attempt to handle the apparent memory leak of exactextract
            currStats = None
            gc.collect()

  0%|          | 0/16528 [00:00<?, ?it/s]C:\Users\olivoshj\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone-3\Lib\site-packages\osgeo\gdal.py:311: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(
100%|██████████| 16528/16528 [10:36<00:00, 25.95it/s]﻿


In [ ]:
# export csv table
arcpy.conversion.ExportTable(
    in_table=input_ranges
    out_table=r"R:\FWL\Arismendi-Lab\Andres\Gilbert_Freshwater_Fish_Analysis\Revised_Analysis_NatureCommunications\Input_datasets\WorldClim_Attributes.csv",
    field_mapping=fr'sci_name "sci_name" true true false 100 Text 0 0,First,#,{input_ranges},sci_name,0,99;bio_1 "bio_1" true true false 4 Float 0 0,First,#,{input_ranges},bio_1,-1,-1;bio_2 "bio_2" true true false 4 Float 0 0,First,#,{input_ranges},bio_2,-1,-1;bio_3 "bio_3" true true false 4 Float 0 0,First,#,{input_ranges},bio_3,-1,-1;bio_4 "bio_4" true true false 4 Float 0 0,First,#,{input_ranges},bio_4,-1,-1;bio_5 "bio_5" true true false 4 Float 0 0,First,#,{input_ranges},bio_5,-1,-1;bio_6 "bio_6" true true false 4 Float 0 0,First,#,{input_ranges},bio_6,-1,-1;bio_7 "bio_7" true true false 4 Float 0 0,First,#,{input_ranges},bio_7,-1,-1;bio_8 "bio_8" true true false 4 Float 0 0,First,#,{input_ranges},bio_8,-1,-1;bio_9 "bio_9" true true false 4 Float 0 0,First,#,{input_ranges},bio_9,-1,-1;bio_10 "bio_10" true true false 4 Float 0 0,First,#,{input_ranges},bio_10,-1,-1;bio_11 "bio_11" true true false 4 Float 0 0,First,#,{input_ranges},bio_11,-1,-1;bio_12 "bio_12" true true false 4 Float 0 0,First,#,{input_ranges},bio_12,-1,-1;bio_13 "bio_13" true true false 4 Float 0 0,First,#,{input_ranges},bio_13,-1,-1;bio_14 "bio_14" true true false 4 Float 0 0,First,#,{input_ranges},bio_14,-1,-1;bio_15 "bio_15" true true false 4 Float 0 0,First,#,{input_ranges},bio_15,-1,-1;bio_16 "bio_16" true true false 4 Float 0 0,First,#,{input_ranges},bio_16,-1,-1;bio_17 "bio_17" true true false 4 Float 0 0,First,#,{input_ranges},bio_17,-1,-1;bio_18 "bio_18" true true false 4 Float 0 0,First,#,{input_ranges},bio_18,-1,-1;bio_19 "bio_19" true true false 4 Float 0 0,First,#,{input_ranges},bio_19,-1,-1'
)

In [ ]:
# delete intermediate files
os.remove(f"{temp_folder}\sqlite.gpkg")